In [1]:
import math
import numpy as np 
import matplotlib.pyplot as plt 

In [2]:
def init(N):
    grid = np.zeros((N,N))
    grid[0] = np.ones(N)
    return grid

In [3]:
def analytic(Y, t, t_max, D):
    t = t * dt
    c = 0
    for i in range(t_max):
        Z_neg = (1 - Y + 2*i) / (2* np.sqrt(D*t))
        Z_pos = (1 + Y + 2*i) / (2* np.sqrt(D*t))
        c = c + math.erfc(Z_neg) - math.erfc(Z_pos)
    return c

1.4 Jacobi iteration

In [5]:
def update_J(Y):
    Y_prev = np.copy(Y)
    for j in range(1, len(Y)-1):
        for i in range(len(Y)):
            if i == 0:
                 Y[j,i] = (1/4) * (Y_prev[j+1,i] + Y_prev[j-1,i]+ Y_prev[j,i+1] +Y_prev[j,-1] -4 * Y_prev[j,i]) 
            elif i == N-1:
                 Y[j,i] = (1/4) * (Y_prev[j+1,i] + Y_prev[j-1,i]+ Y_prev[j,N-1] +Y_prev[j,i-1] -4 * Y_prev[j,i]) 
            else:
                 Y[j,i] = (1/4) * (Y_prev[j+1,i] + Y_prev[j-1,i]+ Y_prev[j,i+1] +Y_prev[j,i-1] -4 * Y_prev[j,i])
    return(Y)  

# we still need to implement the stopping condition

1.5 The Gauss_Seidel teration

In [7]:
def update_GS(Y):
    for j in range(1, len(Y)-1):
        for i in range(len(Y)):
            if i == 0:
                 Y[j,i] = (1/4) * (Y[j+1,i] + Y[j-1,i]+ Y[j,i+1] +Y[j,-1] -4 * Y[j,i]) 
            elif i == N-1:
                 Y[j,i] = (1/4) * (Y[j+1,i] + Y[j-1,i]+ Y[j,N-1] +Y[j,i-1] -4 * Y[j,i]) 
            else:
                 Y[j,i] = (1/4) * (Y[j+1,i] + Y[j-1,i]+ Y[j,i+1] +Y[j,i-1] -4 * Y[j,i])
    return(Y)  

1.6 Successice Over Relaxation

In [ ]:
def update_SOR(Y, w):
    for j in range(1, len(Y)-1):
        for i in range(len(Y)):
            if i == 0:
                 Y[j,i] = (1-w)*Y[j,i] +  (w/4) * (Y[j+1,i] + Y[j-1,i]+ Y[j,i+1] +Y[j,-1] -4 * Y[j,i]) 
            elif i == N-1:
                 Y[j,i] = (1-w)*Y[j,i] +  (w/4) * (Y[j+1,i] + Y[j-1,i]+ Y[j,N-1] +Y[j,i-1] -4 * Y[j,i]) 
            else:
                 Y[j,i] = (1-w)*Y[j,i] +  (w/4) * (Y[j+1,i] + Y[j-1,i]+ Y[j,i+1] +Y[j,i-1] -4 * Y[j,i])
    return(Y)  

w = 1.8
